##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 初学者的 TensorFlow 2.0 教程

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


生成训练数据，暂存于本地，生成结束后再同步到MyDrive：

In [18]:
import matplotlib.image as mpimg  # mpimg 用于读取图片
import numpy as np
from random import random
import math
import os
import cv2
imgs = []
YSTART = 262
XSTART = 162
YLEN = 38
XLEN = 18
LEN = 13 - 4
#DIRS = [[-1,-1],[-1,0],[-1,1],[0,1]]
DIRS = [[-1,0],[1,0],[0,-1],[0,1]]

LINENUM = 30

if not os.path.exists('truth'):
      os.makedirs('truth')
if not os.path.exists('64data'):
      os.makedirs('64data')

for i0 in range(0,30000):
    imgnew = np.ones((XLEN,YLEN,3),dtype=np.uint8)*255
    for i in range(0,LINENUM):
        while True:
            x = int(random() * XLEN)
            y = int(random() * YLEN)
            dip = int(random()*4) 
            di = DIRS[dip]
            cross = 0
            value = int(random()*LEN)+4
            for j in range(value):
                ix = x + di[0] * j
                iy = y + di[1] * j
                if ix < 0 or ix >= XLEN or iy < 0 or iy >= YLEN: break
                if imgnew[ix][iy][0] == 0: cross += 1
            if cross >= 3: continue
            if cross != 1 and random() < 0.5: continue

            adj = 0
            erp = 0
            for p in range(4):
                if p == dip: continue
                for j in range(value):
                    ix = x + DIRS[p][0] + di[0] * j
                    iy = y + DIRS[p][1] + di[1] * j
                    if ix < 0 or ix >= XLEN or iy < 0 or iy >= YLEN: break
                    if imgnew[ix][iy][0] == 0: adj += 1
                    else: adj = 0
                    if adj >= 3:
                        erp = 1
                        break
                if erp == 1:break
            if erp == 1:continue

            for j in range(value):
                ix = x + di[0] * j
                iy = y + di[1] * j
                if ix < 0 or ix >= XLEN or iy < 0 or iy >= YLEN: break
                imgnew[ix][iy][0] = 0
                imgnew[ix][iy][1] = 0
                imgnew[ix][iy][2] = 0
            break  


    mpimg.imsave(r"truth/"+str(i0)+r".jpg", imgnew)

    noize = np.random.normal(-50, 50, (XLEN,YLEN))
    noizes = np.stack((noize, noize, noize),axis = 2)
    imgnoize = np.clip(imgnew + noizes, 0, 255).astype(np.uint8)

    imblur = cv2.blur(imgnoize,(3,3))

    for i in range(8):
        for j in range(8):
            imlarge = cv2.resize(imblur, (YLEN*8,XLEN*8))
            img1 = np.roll(imlarge, i, axis=0)
            img2 = np.roll(img1, j, axis=1)
            imgsmall = cv2.resize(img2, (YLEN,XLEN))

            mpimg.imsave(r"64data/"+str((i*8+j)+64*i0)+r".jpg", imgsmall)
    if(i0%1000==0):print(i0)


0
1000
2000
3000
4000
5000
6000
7000
8000


KeyboardInterrupt: ignored

In [0]:
同步到MyDrive（可能花费很长时间）

In [0]:
!cp -r 64data 'gdrive/My Drive/64data'
!cp -r truth 'gdrive/My Drive/truth'

^C
/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [19]:
import os
print(len(os.listdir('64data')))
#print(len(os.listdir('gdrive/My Drive/64data')))
print(len(os.listdir('truth')))
PIC_NUM = 8000
#print(len(os.listdir('gdrive/My Drive/truth')))


1920000
30000


In [0]:
import cv2

truthpics = [cv2.imread('truth/'+str(i)+'.jpg',cv2.IMREAD_GRAYSCALE)/255.0 for i in range(PIC_NUM)]
datapics = [cv2.imread('64data/'+str(i*64)+'.jpg',cv2.IMREAD_GRAYSCALE)/255.0 for i in range(PIC_NUM)]

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# 安装 TensorFlow
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass

import tensorflow as tf

In [0]:
from matplotlib import pyplot as plt
import cv2
import numpy as np
def similarity(i1, i2):
    im1 = i1>0.5
    im2 = i2>0.5
    return(np.sum((im1*im2)>0)/np.sum((im1+im2)>0))    


def train_generator():
  while True:
        for i in range(int(PIC_NUM*0.8)):
            i1 = truthpics[i]
            i2 = datapics[i]
            yield(np.stack((i1,i2),axis=2),1.0)
        shuffle = np.random.permutation(np.arange(int(PIC_NUM*0.8)))
        for i in range(int(PIC_NUM*0.8)):
            i1 = truthpics[shuffle[i]]
            i2 = datapics[i]
            yield(np.stack((i1,i2),axis=2),similarity(i1,truthpics[i]))
def test_generator():
  while True:
        for i in range(int(PIC_NUM*0.8),PIC_NUM):
            i1 = truthpics[i]
            i2 = datapics[i]
            yield(np.stack((i1,i2),axis=2),1.0)
        shuffle = np.random.permutation(np.arange(int(PIC_NUM*0.8),PIC_NUM))
        for i in range(int(PIC_NUM*0.8),PIC_NUM):
            i1 = truthpics[shuffle[i]]
            i2 = datapics[i]         
            yield(np.stack((i1,i2),axis=2),similarity(i1,truthpics[i]))
train_dataset = tf.data.Dataset.from_generator(train_generator, (tf.float32, tf.float32), (tf.TensorShape([18,38,2]),tf.TensorShape([])))
train_dataset=train_dataset.batch(100).shuffle(100)
test_dataset = tf.data.Dataset.from_generator(test_generator, (tf.float32, tf.float32), (tf.TensorShape([18,38,2]),tf.TensorShape([])))
test_dataset=test_dataset.batch(100).shuffle(100)

载入并准备好 [MNIST 数据集](http://yann.lecun.com/exdb/mnist/)。将样本从整数转换为浮点数：

将模型的各层堆叠起来，以搭建 `tf.keras.Sequential` 模型。为训练选择优化器和损失函数：

In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(18, 38, 2)),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
  tf.keras.layers.MaxPooling2D((2, 2)),
  tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
  tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam',
              loss='mse',
              metrics=['mae'])

训练并验证模型：

In [0]:
model.fit(train_dataset,epochs=2)

Epoch 1/2
 117502/Unknown - 2725s 23ms/step - loss: 0.0401 - mae: 0.1922

In [12]:

model.evaluate(test_dataset, verbose=2)

UnknownError: ignored

现在，这个照片分类器的准确度已经达到 98%。想要了解更多，请阅读 [TensorFlow 教程](https://www.tensorflow.org/tutorials/)。